## Oppstart

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import dhlab.nbtext as nb
import dhlab.token_map as tm
from dhlab.nbtext import get_urn, make_graph, get_urn, relaterte_ord, navn, totals
import dhlab.graph_networkx_louvain as gnl
import pandas as pd
import networkx as nx

%matplotlib inline

plot = lambda x,  lw=3, alpha= 0.7: x.plot(figsize=(15,6), lw=lw, alpha=alpha)


In [3]:
from dhlab.module_update import update, css
css()

# Hjelpekommandoer

I den her notebooken er det satt opp kode for å telle opp navn i et korpus. Både listen med navn og korpuset må foreligge som excelfiler. Noen av kommandoene vil bli lagt til `token_map`-modulen 

* `tell_navn` teller opp navnene på tvers av korpuset 
* `korpus_oppføring` finner data i korpuset
* `vis_bok` viser navn i en spesifikk bok (benytter oppførings-kommandoen)
* `aggregate_pr_col` aggregerer navnedata over kolonner.

In [37]:
import pandas as pd
from nbtext import metadata

def alle_korpusnavn(korpus):
    urner = nb.pure_urn(korpus)
    alle_navn = tm.combine_names(tm.corpus_names(urner))
    return alle_navn

def tell_navn(korpus, token_map):
    res = dict()
    for urn in nb.pure_urn(korpus['urn']):
        try:
            res[urn] = tm.count_name_strings(str(urn), token_map)
            res[urn].columns = [str(urn)]
        except:
            print("Problemer med å telle fra: ", urn)
    return pd.concat([res[u] for u in res], axis=1)

def korpus_oppføring(df, text, column='tittel'):
    return nb.pure_urn(df[df[column].str.contains(text)]['urn'])

def add_metadata(korpus, how='inner'):
    f = nb.frame([(z[0], '_'.join([str(x) for x in z[2:4]])) for z in nb.metadata(korpus['urn'])], 'urn metadata'.split())
    f = f.astype({'urn':'str'})
    return korpus.merge(f, on = 'urn', how = how)
    
vis_bok = lambda x, column = 'tittel': nb.frame_sort(nb.frame(opptelling[korpus_oppføring(korpus, x, column)[0]].dropna(), nb.metadata(korpus_oppføring(korpus, x, column)[0])[0][3]))

    

def bygg_allgraf(korpus):
    graphs = dict()
    for u in korpus.urn:
        graphs[u] = tm.character_network(u, redigerte_navn)
    df_graphs = [nx.to_pandas_adjacency(graphs[x]) for x in graphs]
    g = df_graphs[0]
    for z in df_graphs[1:]:
        g = g.add(z, fill_value=0).fillna(0)

    total_graph = nx.from_pandas_adjacency(g)
    return total_graph

# Sammenlign kollokasjoner
compare = lambda coll, korpus_sum, freq=1: nb.frame_sort(nb.frame(coll[coll >= freq][0]/korpus_sum[0]))

## Navn i bøker


Vi søker etter navn i en bok, og en serie av bøker.

Kommandoene er:
1. `names` henter et sett med forslag til navn, fra nbtext
1. `show_names` gir en oversikt over funnene, fra token_map
1. `names_to_token_map_file` lagrer navneforslagene til en excelfil (eller csv) for redigering (token_map)
1. `read_token_map_file` henter redigert fil tilbake for analyse (token_map)
1. `character_network` lager graf for navnene (token_map)
1. `show_graph`  tegner grafen (fra graph_networkx_louvain)
1. `show_communities` viser clustre i grafen (graph_networkx_louvain)



## Korpus

Hent korpus fra excelfil. Det er korpuset det skal telles opp fra. Korpuset som genererer navn kan godt være forskjellig fra der det telles.

In [5]:
korpus = nb.restore_metadata_from_excel('c:/users/larsj/downloads/Hamsunkorpus, kronologisk_ed.xls')
korpus

,urn,forf,år,tittel,undertittel,forlag,sjanger,sesamid,språk,oversatt
0,2008112113001,"Hamsun, Knut",1877,Den Gaadefulde,en Kjærlighedshistorie fra Nordland,Urdal,unknown,05b5fb8e2ffab70be305373f9857ee85,nob,NaN
1,2009061013011,"Hamsun, Knut",1878,Bjørger,fortælling,[K. Hamsund];Alb.Fr. Knudsen,unknown,709614fcb48740925154af4130d7a26c,nob,NaN
2,2009061013012,"Hamsun, Knut",1878,Et gjensyn,NaN,[s.n.];Alb. Fr. Knudsen,unknown,233ad96c3f3bedf1739ceb29a81534d4,nob,NaN
3,2009061012004,"Hamsun, Knut",1889,Fra det moderne Amerikas Aandsliv,NaN,Philipsen,unknown,949c8474bef0068c4d97f02c2f646284,nob,NaN
4,2009042313001,"Hamsun, Knut",1889,Lars Oftedal,Udkast,Litleré,unknown,9b6cc05c90fa5a5ccc27f018e2c27281,nob,NaN
5,2015051929001,"Hamsun, Knut",1890,Sult,NaN,Philipsen,novel,b4a0a4c7765fa29eb4a1ba910405806e,nob,NaN
6,2008123012002,"Hamsun, Knut",1892,Mysterier,Roman,Philipsen,unknown,9f41e621267dd5475db9b7e45cd00367,nob,NaN
7,2008123012003,"Hamsun, Knut",1893,Redaktør Lynge,Roman,Philipsen,fiction,e09ea1ee062affda9ab9fdfde810517c,nob,NaN
8,2008123010002,"Hamsun, Knut",1893,Ny Jord,Roman,Philipsen,unknown,aca8b7840508f729920970a51fea6c05,nob,NaN
9,2008082710002,"Hamsun, Knut",1894,Pan,af Løjtnant Thomas Glahns Papirer,Philipsen,unknown,8dab706553bacde19fa1a30c06896464,nob,NaN


## Sjekk innholdet

Gjør noen søk for å se at alt er ok. Konkordanser er fine til det. 

In [6]:
nb.urn_concordance(word = 'Benoni', urns = korpus) 

"Rosa, Hamsun, Knut, 1908",leved ogsaa den gode Mand,Benoni,Hartvig-
"Rosa, Hamsun, Knut, 1908",Middagstiden gik jeg derimod til,Benoni,Hartvigsens Hus og fremstilled mig
"Rosa, Hamsun, Knut, 1908",det Dem som er hos,Benoni,Hart vigsen ? spurte han
"Rosa, Hamsun, Knut, 1908",", det var nu han",Benoni,og jeg som betalte ham
"Rosa, Hamsun, Knut, 1908","mit hele Navn , for",Benoni,det er jeg døbt i
"Benoni, Hamsun, Knut, 1908",foruden dette er det bare,Benoni,som har sin Gang her
"Benoni, Hamsun, Knut, 1908",Sommer og Vinter . Og,Benoni,gaar i tørt Vejr og
"Benoni, Hamsun, Knut, 1908",None,Benoni,han er Fisker som alle
"Benoni, Hamsun, Knut, 1908","hvert Kvartal, og derfor er",Benoni,en Støver og en Brand
"Benoni, Hamsun, Knut, 1908",af Rigdom paa Vejene .,Benoni,derimod kom uforanderlig Aar efter
"Vandrer spiller med Sordin, Hamsun, Knut, 1909",None,Benoni,", Roman 1908 5.50"


# Referanser for kollokasjoner

In [14]:
# Finn totalen for korpuset
korpus_sum = nb.get_aggregated_corpus(nb.pure_urn(korpus))

# Henter ut de 50 000 mest frekvente ordene fra bokhylla
bokhylla_sum = nb.frame(nb.totals(50000))

## Lag kollokasjoner 

Kommandoen er `urn_coll()`. Resultatet fra kommandoen er en frekvensliste for søkeordet. Frekvenslisten kan analyseres med hensyn til korpus eller med hensyn til bokhylla. Eksperimenter med parametrene `before` og `after`. I resultatene ser du mønsteret x.head(10) som betyr at de 10 første skal vises, selv om det er mer.

In [51]:
coll = nb.urn_coll("Benoni", urns = nb.pure_urn(korpus), before=10, after=10)
coll.head(10)

,0
.,1055
",",549
:,148
?,139
Mack,131
Det,100
!,87
Men,68
Han,66
Jeg,62


## Sammenlign

Kollokasjonene kan testes mot korpus og bokhylla, benytt kommando `compare(kollokasjone, sammenligning, frekvens)` der `sammenligning` er data det sammenlignes mot - i vårt tilfelle er det de to variablene `korpus_sum` og `bokhylla_sum`, og `frekvens` er minimum antall forekomster i kollokasjonen. Grei for å fjerne ord - sjeldne ord har en tendens til å boble opp, men er gjerne ikke så interessante. Prøv med litt forskjellig verdier og se hva som dukker opp. 

In [56]:
# Mot bokhylla - legg merke til forskjellen fra å sammenligne med korpuset
compare(coll, bokhylla_sum, 1).head(20)

,0
Mack,0.002044
Benoni,0.001719
Dokker,0.001314
Daler,0.000415
Sild,0.000303
Rosa,0.000294
Million,0.000207
Hugh,0.000200
Bygden,0.000200
Jaja,0.000187


In [60]:
compare(coll, korpus_sum, 3).head(20)

,0
Mageløst,0.600000
Pantebrevet,0.500000
Bergensturen,0.500000
Klenodierne,0.428571
Krambodsvend,0.361111
1908,0.357143
Malm,0.333333
Klipfisken,0.300000
Marelius,0.290323
Herligheder,0.272727
